# SV Distribution
## Goal
The goal of this code is to create csv files with the distances of each synaptic vesicle (SV) to the nearest point on the active zone (AZ) line. This should be separated by condition.

## What you need to run this code: 
1. You need a SV.zip file in every synapse folder. This zip file contains ROIs for each SV in that synapse. There can only be one zip file in that folder that ends with SV.zip. Ensure that is your final SV count.
2. You need an AZ.txt file. This is explained below:
    * Open synapse image in Fiji and draw a freehand line along the AZ (electron dense PSD)
    * Select your AZ line --> File--> Save as...--> XY Coordinates. 
    * Save in Syn__ folder as Syn__AZ.txt
        * This file is a list of some coordinates on your line, NOT all of them. The code will fix that. 
3. You need a main folder, mine was called Synapse Files. If yours is different, you need to change the code in "call previous function while running through folders" cell in Synapse_files = Path('**YOUR FOLDER NAME**/')
    * Under my main Synapse Files folder, I have separate folders for each condition. Within those condition folders, are the folders of each synapse corresponding to that condition.

## Important Note on AZ file format
Save your AZ.txt files without a scale (draw your AZ line on your image, save as a normal ROI file, then exit out of the image. This will delete any scale. Then save your AZ line as X,Y coordinates, and label the file Syn__AZ.txt

In [1]:
#Import packages 
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import zipfile
import seaborn as sns
import time
from PIL import Image
sns.set()

## Define a function to find the shortest distance between all SVs and the AZ line for each synapse.
This function is called vesicle_distances. It takes one argument, which is is the path for our synapse folders. This does not run the function yet, we will do that in the next cell.


In [2]:
def vesicle_distances(syn_path):
#Code to turn AZ line into X,Y coordinates
    img_conversion = 0.272 #px/nm. 
    
    #Define our path as anything in folders with an AZ.txt in it    
    az_txt = syn_path / Path(syn_path.name + 'AZ.txt')
    
    df_raw = pd.read_table(str(az_txt), header=None)
    
    #Read the X,Y coordinates from the AZ.txt file and convert to pixels
    df = np.round(pd.read_table(str(az_txt), header=None)*img_conversion, decimals=0)
    df_diff = df.diff()
    
    #Finding Euclidean distance
    df_ed = np.sqrt(df_diff.iloc[:,0]**2 +df_diff.iloc[:,1]**2) 
        #This code will only give you some pixel X,Y coordinates, not everything on that line

    #Get the distances of all X,Y coordinates on that line:
    #If the distance between 2 pixel points is more than 1, fill in the blanks by 1 to make a continuous line
    x_fin, y_fin = [], [] 
    for i in range(len(df)): 
        x, y = df.iloc[i,:]
        x_fin.append(x)
        y_fin.append(y)
        if i == len(df) - 1:
            continue
        if df_ed.iloc[i+1] < 1.1:
            continue
        x_fin += list(np.linspace(x, df.iloc[i+1,0], int(df_ed[i+1])))
        y_fin += list(np.linspace(y, df.iloc[i+1,1], int(df_ed[i+1])))
    
    #Create an array for all X,Y coordinates along your AZ line
    L = np.array([x_fin,y_fin]).T    

    #Self check: If you want to see all of your X,Y values of your line now, uncomment Ltable_pd
    Ltabletest = {'X values':x_fin, 'Y values':y_fin}
    Ltable_pd = pd.DataFrame(Ltabletest)
    Ltable_pd
        #Notice now none of the values are more than 1px apart
        
 #Code to extract X,Y coordinates from SV ROIs      
    ##Create path where all ROIs are    
    vesicle_path = syn_path / Path(syn_path.name + 'SV')
    assert vesicle_path.is_dir()

    #Make an array called names where you will put the X,Y coordinates from each ROI file name
    names = []
    for file in vesicle_path.iterdir():
        names.append(file.stem)

    #Create arrays for X and Y coordinates for each SV    
    names_split = [name.split('-')[0:2] for name in names]
    v_x, v_y = [], []
    for y_str, x_str in np.unique(names_split, axis=0):
        v_x.append(int(x_str))
        v_y.append(int(y_str))
        
    #Create an array for all X,Y coordinates of all SVs
    V = np.array([v_x,v_y]).T 

    #Self check: If you want to see all of your X,Y values of your SVs, uncomment Vtable_pd
    Vtabletest = {'X values':v_x, 'Y values':v_y}
    Vtable_pd = pd.DataFrame(Vtabletest)
    #print(Vtable_pd)

#Code to find the distance from each ROI in V(vesicles) to any one point in L(AZline)
    min_dist= []
    for v in V:
        d_v = []
        for p in L:
            d_v.append(distance(v,p))
        min_dist.append(np.min(d_v))
        
#KF modification start - 050621
#Code to find the distance from each ROI in V(vesicles) to every other vesicle (V) 

    min_dist_sv= []
    for v in V:
        d_sv_v = []
        for i in V:
            if sv_distance(v,i) != 0:
                d_sv_v.append(sv_distance(v,i)) 
        min_dist_sv.append(np.min(d_sv_v))
        
        
    #Create an array of the shortest distance of each SV ROI to the coordinates of the line, convert from px to nm
    SVdistances = np.array(min_dist)/0.272
    SVdistances2 = np.array(min_dist_sv)/0.272 
    
    return SVdistances, min_dist, L, V, min_dist_sv, SVdistances2

#KF modification end


## Call the previous function while running through folders.
This will go through our folders, add in an unzipped SV folder, call our previous vesicle_distance function for each Synfolder, and save our files.

In [4]:
#Define the standard distance formula. v stands for vesicle, p stands for pixel
def distance(v,p):
    return np.sqrt((v[0]-p[0])**2 +(v[1]-p[1])**2)
#define neighbor distance formula, V1 is x values, V2 is y values of SV coordinates
def sv_distance(v,i):
    return np.sqrt((v[0]-i[0])**2 +(v[1]-i[1])**2)

#Dig through folders, locate all SV.zip files and AZ.txt files
Synapse_files = Path('/Users/kaitlynefouke/Desktop/Synapse Files ConIgG')
assert Synapse_files.exists()

for condition in Synapse_files.iterdir():
    #print(condition)
    # Subselect for cond we want. If you want to do all conditions at once, comment these out.
    if condition != Path('/Users/kaitlynefouke/Desktop/Synapse Files ConIgG/ConConIgG'):
        if condition != Path('/Users/kaitlynefouke/Desktop/Synapse Files ConIgG/HighConIgG'):
            if condition != Path('/Users/kaitlynefouke/Desktop/Synapse Files ConIgG/LowConIgG'):
                continue
    #print(condition)
    for Synfolders in condition.iterdir():
        if Synfolders != Path(condition/'.DS_Store'):   
            print(Synfolders)
            syn_sv = Synfolders / Path(Synfolders.name + 'SV.zip')
        #print(syn_sv)
        #Self check: Do they exist? True, then continues
            assert syn_sv.exists()
            
            syn_az = Synfolders / Path(Synfolders.name + 'AZ.txt')
        #Self check: Do they exist? True, then continues
            assert syn_az.exists()
        
        #Unzip all SV.zip files in our Synfolders. If unzipped folder does not exist, make one.
            syn_sv_unzipped = Synfolders / syn_sv.stem 

            if not syn_sv_unzipped.exists():
                syn_sv_unzipped.mkdir()
            if not len(list(syn_sv_unzipped.iterdir())):
                print('Extracting SVs')
                syn_sv_zip = zipfile.ZipFile(str(syn_sv))
                syn_sv_zip.extractall(str(syn_sv_unzipped))
                
        #Save the 6 data files in their correct folders - these have to be in the correct order as listed above in the function def
            out_file_names = ['sv_dist.txt', 'pix_dist.txt', 'L.txt', 'V.txt', 'sv_sv_pix_dist.txt', 'sv_sv_dist.txt']
            #if not np.all([Path(Synfolders / out_file).exists() for out_file in out_file_names]):

        #Call our vesicle_distances function to create distances for each Synfolder
            vesicle_returns = vesicle_distances(Synfolders)

            for arr, name in zip(vesicle_returns, out_file_names):
                vc_ret_path = Path(Synfolders / name)
                    #if not vc_ret_path.exists():
                np.savetxt(str(vc_ret_path), arr)
                        
        #Shows us progress while it's working, lets us know when it has completed for each condition             
                print('Completed {0}!'.format(Synfolders))
            else:
                print('Found all distance files, skipping {0}...'.format(Synfolders))
            
    print('Completed synapses for condition {0}'.format(condition.name))

/Users/kaitlynefouke/Desktop/Synapse Files ConIgG/LowConIgG/Syn75
Completed /Users/kaitlynefouke/Desktop/Synapse Files ConIgG/LowConIgG/Syn75!
Completed /Users/kaitlynefouke/Desktop/Synapse Files ConIgG/LowConIgG/Syn75!
Completed /Users/kaitlynefouke/Desktop/Synapse Files ConIgG/LowConIgG/Syn75!
Completed /Users/kaitlynefouke/Desktop/Synapse Files ConIgG/LowConIgG/Syn75!
Completed /Users/kaitlynefouke/Desktop/Synapse Files ConIgG/LowConIgG/Syn75!
Completed /Users/kaitlynefouke/Desktop/Synapse Files ConIgG/LowConIgG/Syn75!
Found all distance files, skipping /Users/kaitlynefouke/Desktop/Synapse Files ConIgG/LowConIgG/Syn75...
/Users/kaitlynefouke/Desktop/Synapse Files ConIgG/LowConIgG/Syn72
Completed /Users/kaitlynefouke/Desktop/Synapse Files ConIgG/LowConIgG/Syn72!
Completed /Users/kaitlynefouke/Desktop/Synapse Files ConIgG/LowConIgG/Syn72!
Completed /Users/kaitlynefouke/Desktop/Synapse Files ConIgG/LowConIgG/Syn72!
Completed /Users/kaitlynefouke/Desktop/Synapse Files ConIgG/LowConIgG/S

## Create/save files that separates synapse distance files by condition.
You should have one excel sheet for each condition, and in it you will have synapse specific data.

In [5]:
#Put synapse distance files into Excel sheets
Synapse_files = Path('/Users/kaitlynefouke/Desktop/Synapse Files ConIgG')
distance_directory = Path('/Users/kaitlynefouke/Desktop/Synapse Files ConIgG/Vesicle Distance Files')
if not distance_directory.exists():
    distance_directory.mkdir()

assert Synapse_files.exists()
for condition in Synapse_files.iterdir():
    if condition != Path('/Users/kaitlynefouke/Desktop/Synapse Files ConIgG/ConConIgG'):
        if condition != Path('/Users/kaitlynefouke/Desktop/Synapse Files ConIgG/HighConIgG'):
            if condition != Path('/Users/kaitlynefouke/Desktop/Synapse Files ConIgG/LowConIgG'):
                continue
    df = pd.DataFrame()
    df2 = pd.DataFrame()
    print(condition)
    for Synfolders in condition.iterdir():
        if Synfolders != Path(condition/'.DS_Store'):
            df_dist = pd.DataFrame(np.loadtxt(str(Synfolders / 'sv_dist.txt')),
                                    columns=[Synfolders.name])
            df = pd.concat([df, df_dist], sort=False, axis=1)
            df_sv_dist = pd.DataFrame(np.loadtxt(str(Synfolders / 'sv_sv_dist.txt')),
                                    columns=[Synfolders.name])
            df2 = pd.concat([df2, df_sv_dist], sort=False, axis=1)

    df_path = distance_directory / '{0}_distances.csv'.format(condition.name)
    df.to_csv(str(df_path))
    print('Created new distance file: {0}'.format(df_path))
    
    df2_path = distance_directory / '{0}_neighbordistances.csv'.format(condition.name)
    df2.to_csv(str(df2_path))
    print('Created new neighbor distance file: {0}'.format(df2_path))

            
            

/Users/kaitlynefouke/Desktop/Synapse Files ConIgG/LowConIgG
Created new distance file: /Users/kaitlynefouke/Desktop/Synapse Files ConIgG/Vesicle Distance Files/LowConIgG_distances.csv
Created new neighbor distance file: /Users/kaitlynefouke/Desktop/Synapse Files ConIgG/Vesicle Distance Files/LowConIgG_neighbordistances.csv
/Users/kaitlynefouke/Desktop/Synapse Files ConIgG/HighConIgG
Created new distance file: /Users/kaitlynefouke/Desktop/Synapse Files ConIgG/Vesicle Distance Files/HighConIgG_distances.csv
Created new neighbor distance file: /Users/kaitlynefouke/Desktop/Synapse Files ConIgG/Vesicle Distance Files/HighConIgG_neighbordistances.csv
/Users/kaitlynefouke/Desktop/Synapse Files ConIgG/ConConIgG
Created new distance file: /Users/kaitlynefouke/Desktop/Synapse Files ConIgG/Vesicle Distance Files/ConConIgG_distances.csv
Created new neighbor distance file: /Users/kaitlynefouke/Desktop/Synapse Files ConIgG/Vesicle Distance Files/ConConIgG_neighbordistances.csv


In [6]:
from collections.abc import Iterable

def as_list(obj, length=None, tp=None, iter_to_list=True):
    """
    Force an argument to be a list, optionally of a given length, optionally
    with all elements cast to a given type if not None.
    Parameters
    ---------
    obj : Object
        The obj we want to convert to a list.
    length : int or None, optional
        Length of new list. Applies if the inputted obj is not an iterable and
        iter_to_list is false.
    tp : type, optional
        Type to cast the values inside the list as.
    iter_to_list : bool, optional
        Determines if we should cast an iterable (not str) obj as a list or to
        enclose it in one.
    Returns
    -------
    obj : list
        The object enclosed or cast as a list.
    """
    # If the obj is None, return empty list or fixed-length list of Nones
    if obj is None:
        if length is None:
            return []
        return [None] * length
    
    # If it is already a list do nothing
    elif isinstance(obj, list):
        pass

    # If it is an iterable (and not str), convert it to a list
    elif isiterable(obj) and iter_to_list:
        obj = list(obj)
        
    # Otherwise, just enclose in a list making it the inputted length
    else:
        try:
            obj = [obj] * length
        except TypeError:
            obj = [obj]
        
    # Cast to type; Let exceptions here bubble up to the top.
    if tp is not None:
        obj = [tp(o) for o in obj]
    return obj

def isiterable(obj):
    """
    Function that determines if an object is an iterable, not including 
    str.
    Parameters
    ----------
    obj : object
        Object to test if it is an iterable.
    Returns
    -------
    bool : bool
        True if the obj is an iterable, False if not.
    """
    if isinstance(obj, str):
        return False
    else:
        return isinstance(obj, Iterable)
    
def _flatten(inp_iter):
    """
    Recursively iterate through values in nested iterables.
    Parameters
    ----------
    inp_iter : iterable
        The iterable to flatten.
    Returns
    -------
    value : object
        The contents of the iterable
    """
    for val in inp_iter:
        if isiterable(val):
            for ival in _flatten(val):
                yield ival
        else:
            yield val
            
def flatten(inp_iter):
    """
    Returns a flattened list of the inputted iterable.
    Parameters
    ----------
    inp_iter : iterable
        The iterable to flatten.
    Returns
    -------
    flattened_iter : list
        The contents of the iterable as a flat list
    """
    return list(_flatten(inp_iter))

## Compress all distances from all synapses into just one column for each condition. 
This will generate and save a file which only has as many columns as you do conditions, and all of the distances for one condition will be in the same column.

In [7]:
Synapse_files = Path('/Users/kaitlynefouke/Desktop/Synapse Files ConIgG')
distance_directory = Path('/Users/kaitlynefouke/Desktop/Synapse Files ConIgG/Vesicle Distance Files')
if not distance_directory.exists():
    distance_directory.mkdir()
    
assert Synapse_files.exists()

####HK test
for condition in Synapse_files.iterdir():
    if condition == Path(Synapse_files/'.DS_Store'):
        os.remove(condition)
        
for condition in Synapse_files.iterdir():
    if condition != Path('/Users/kaitlynefouke/Desktop/Synapse Files ConIgG/ConConIgG'):
        if condition != Path('/Users/kaitlynefouke/Desktop/Synapse Files ConIgG/HighConIgG'):
            if condition != Path('/Users/kaitlynefouke/Desktop/Synapse Files ConIgG/LowConIgG'):
                continue
    for Synfolders in condition.iterdir():
        if Synfolders == Path(condition/'.DS_Store'):
            os.remove(Synfolders)
            
####HK test

df = pd.DataFrame()
df2 = pd.DataFrame()

for condition in Synapse_files.iterdir():
    if condition != Path('/Users/kaitlynefouke/Desktop/Synapse Files ConIgG/ConConIgG'):
        if condition != Path('/Users/kaitlynefouke/Desktop/Synapse Files ConIgG/HighConIgG'):
            if condition != Path('/Users/kaitlynefouke/Desktop/Synapse Files ConIgG/LowConIgG'):
                continue
            
    df_dist = pd.DataFrame(
        flatten([np.loadtxt(str(Synfolders/'sv_dist.txt')) 
                        for Synfolders in condition.iterdir()]),
        columns=[condition.name])
        
    df_sv_dist = pd.DataFrame(
        flatten([np.loadtxt(str(Synfolders/'sv_sv_dist.txt')) 
                        for Synfolders in condition.iterdir()]),
        columns=[condition.name])

    df = pd.concat([df, df_dist], sort=False, axis=1)
    df2 = pd.concat([df2, df_sv_dist], sort=False, axis=1)

df.to_csv(str(distance_directory / 'All_conditions.csv'))
df2.to_csv(str(distance_directory / 'All_conditions_neighbors.csv'))

## Analysis is done! 
Check under your directory and ensure that you have all excel sheets. There should be a distance excel sheet for each condition containing synapse specific information, and then an excel sheet 'All_conditions' that has the compiled data for each condition. 
